This Notebook is used to analyze and generate a rendering for the first 4 scores of Ludus Musicalis by Roman Haubenstock-Ramati.

The scores physical size is ~20x20 cm (?) (square aspect ratio).

They can be read from 4 different directions, and can be performed simultaneously or in sequence.

The companion document explain the meaning of the symbols and how the pieces should be performed.

In [ ]:
# Profiling instructions
# pip install line_profiler
# %%prun -s cumulative

# pip install memory_profiler
#%load_ext memory_profiler
#%reload_ext memory_profiler
#%mprun

#%%memit -r 1
# Profile a cell by running it _once_
# %%memit MUST be the first line in a cell, even before comments...

In [ ]:
#%%memit -r 1
# First import the MTM package and check its version: 
# pip install multi-template-matching
# Should be 2.0.1 as 2.0.0 had a bug
# It fails if you already have OpenCV (remove it first)
import MTM
from MTM import drawBoxesOnRGB, matchTemplates
print("MTM version: ", MTM.__version__)

import os
import platform

# For parallelizing long computations (where possible)
import concurrent.futures
import multiprocessing

# But we need OpenCV anyway for image processing so we must reinstall it after: 
# pip install opencv-python
import cv2
from scipy.ndimage import rotate
import numpy as np
# pip install matplotlib
import matplotlib.pyplot as plt
import skimage as ski

# For the gamma curve
# pip install splines
import splines

# PyWebView is used for dialog boxes: 
# pip install pywebview
import webview

# mido is used to generate midi files and messages
# On Win: pip install mido[ports-rtmidi]
# On Mac: pip install python-rtmidi 
#         pip install mido
from mido import MetaMessage, Message, MidiFile, MidiTrack

# For creating scores
# pip install music21
import music21
# Music 21 does not recognize MuseScore 4 yet, set the correct location
# On Windows:
# music21.environment.set('musescoreDirectPNGPath', 'C:\\Program Files\\MuseScore 4\\bin\\MuseScore4.exe')
# On Mac:
# TODO

# With PyWebView we prepare the file dialog to show only the supported image types.
def webview_file_dialog():
    file = None
    file_types = ('Image Files (*.bmp;*.jpg;*.gif)', 'All files (*.*)')
    def open_file_dialog(w):
        nonlocal file
        try:
            file = w.create_file_dialog(webview.OPEN_DIALOG, allow_multiple=False, file_types=file_types)[0]
        except TypeError:
            pass  # User exited file dialog without picking
        finally:
            w.destroy()
    window = webview.create_window("", hidden=True)
    webview.start(open_file_dialog, window)
    # file will either be a string or None
    return file

# Custom image scaling function
def scale_image(image, percent, maxwh):
    max_width = maxwh[1]
    max_height = maxwh[0]
    max_percent_width = max_width / image.shape[1] * 100
    max_percent_height = max_height / image.shape[0] * 100
    max_percent = 0
    if max_percent_width < max_percent_height:
        max_percent = max_percent_width
    else:
        max_percent = max_percent_height
    if percent > max_percent:
        percent = max_percent
    width = int(image.shape[1] * percent / 100)
    height = int(image.shape[0] * percent / 100)
    result = cv2.resize(image, (width, height), interpolation=cv2.INTER_AREA)
    return result, percent

# Get dimensions of a window. We are working with fullscreen windows on the main display to simplify the process.
def get_screen_dimensions(window_name="Gamma Correction Preview"):
    import platform
    import cv2
    if platform.system() != "Darwin":
        # Get the dimensions of the specified window.
        (_, _, screen_width, screen_height) = cv2.getWindowImageRect(window_name)
    else:
        # On macOS, use AppKit instead.
        import AppKit
        main_screen = AppKit.NSScreen.mainScreen()
        frame = main_screen.frame()
        screen_width = int(frame.size.width)
        screen_height = int(frame.size.height)
    return screen_width, screen_height

# Compute the LUT using a spline defined only on [low_val, high_val]
# The mapping is: low_val -> 0, mid -> 128, high_val -> 255.
def create_custom_LUT(low, mid_pt, high):
    LUT = np.zeros(256, dtype=np.uint8)
    # Compute the spline for x between low and high.
    spline = splines.CatmullRom([0, 128, 255], [low, mid_pt, high])
    x_range = np.arange(low, high + 1)
    LUT[low:high + 1] = np.clip(spline.evaluate(x_range), 0, 255).astype(np.uint8)
    LUT[:low] = 0
    LUT[high + 1:] = 255
    return LUT

# Function to create a custom LUT for gamma correction.
def interactive_gamma_correction(musicSearch):
    """
    Opens a fullscreen interactive window that lets the user adjust the gamma
    correction using independent controls for the midpoint and the spread.
    
    Controls:
      W/S: Increase/decrease the midpoint.
      Q/A: Decrease/increase the spread (i.e. tighten or widen the mapping range).
      T: Toggle gamma correction on/off.
      ENTER/SPACE: Confirm selection.
      
    When gamma correction is toggled off, the preview displays the original image.
    
    Returns:
       The processed image: if gamma correction is active, the gamma-corrected image,
       otherwise the original image.
    """
    # Define parameters for the controls.
    mid = 127             # Starting midpoint value.
    spread = 127           # Starting half-range so that low = 0, high = 254.
    delta_mid = 1         # Adjustment step for midpoint.
    delta_spread = 1      # Adjustment step for spread.
    min_spread = 1        # Minimum spread value.
    deactivated = False   # Flag to indicate gamma correction is off.

    # Create fullscreen window for preview.
    window_name = "Gamma Correction Preview"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.startWindowThread()

    # Main interactive loop.
    while True:
        # Compute the dependent low and high from mid and spread.
        low_val = mid - spread
        high_val = mid + spread

        # Create the LUT only if gamma correction is active.
        if not deactivated:
            def create_custom_LUT(low, mid_pt, high):
                LUT = np.zeros(256, dtype=np.uint8)
                spline = splines.CatmullRom([0, 128, 255], [low, mid_pt, high])
                x_range = np.arange(low, high + 1)
                LUT[low:high + 1] = np.clip(spline.evaluate(x_range), 0, 255).astype(np.uint8)
                LUT[:low] = 0
                LUT[high + 1:] = 255
                return LUT
            LUT = create_custom_LUT(low_val, mid, high_val)
        
        screen_width, screen_height = get_screen_dimensions(window_name)
        # Build curve points by mapping the LUT if active.
        pts = []
        if not deactivated:
            for i in range(256):
                x = int(i / 255 * (screen_width - 1))
                y = int(screen_height - 1 - (LUT[i] / 255.0 * (screen_height - 1)))
                pts.append((x, y))
            pts = np.array(pts, np.int32).reshape((-1, 1, 2))
        
        # Create preview.
        if deactivated:
            preview = musicSearch.copy()
        else:
            preview = cv2.LUT(musicSearch, LUT)

        # Build overlay text.
        if deactivated:
            overlay_text = ("Gamma Correction DISABLED    "
                            "[T: enable correction]    [ENTER/SPACE: confirm]")
        else:
            overlay_text = (f"Mid: {mid}, Spread: {spread}, Low: {low_val}, High: {high_val}    "
                            "[W/S: move mid]    [Q: tighten, A: widen]    "
                            "[T: disable correction]    [ENTER/SPACE: confirm]")
        
        cv2.putText(preview, overlay_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX,
                    0.8, (0, 255, 0), 2, cv2.LINE_AA)

        # Draw the curve if gamma correction is active.
        if not deactivated:
            cv2.polylines(preview, [pts], isClosed=False, color=(0, 255, 0), thickness=2)

        # Show preview.
        cv2.imshow(window_name, preview)
        key = cv2.waitKey(0) & 0xFF

        # Adjust controls based on key pressed.
        if key == ord('w'):
            # Increase the midpoint, ensuring mid + spread <= 255.
            mid = min(mid + delta_mid, 255 - spread)
        elif key == ord('s'):
            # Decrease the midpoint, ensuring mid - spread >= 0.
            mid = max(mid - delta_mid, spread)
        elif key == ord('q'):
            # Tighten: reduce spread if possible.
            spread = max(min_spread, spread - delta_spread)
        elif key == ord('a'):
            # Widen: increase spread ensuring bounds.
            spread = min(spread + delta_spread, min(mid, 255 - mid))
        elif key == ord('t'):
            # Toggle gamma correction.
            deactivated = not deactivated
        elif key in [13, 32]:
            # Confirm selection.
            break

    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)

    # Return the final image: if deactivated, original; else, apply final LUT.
    if deactivated:
        return musicSearch
    else:
        return cv2.LUT(musicSearch, LUT)

We present the user with the choice of analyzing one of the first 4 scores, supplying another image, or acquiring an image with a webcam.

In [ ]:
#%%memit -r 1
valid_scores = {1, 2, 3, 4, 5, 6}

while True:
    user_input = input("Enter the score number that you want to analyze (1-2 Punkte, 3-4 Pizzicati, 5 Other files, 6 Camera) [type 'q' to quit]: ")
    if user_input.lower() in ["q", "quit"]:
        raise SystemExit("User requested exit.")

    try:
        scoreNumber = int(user_input)
        if scoreNumber in valid_scores:
            break
        else:
            print("Invalid score number. Please enter a number between 1 and 6.")
    except ValueError:
        print("Invalid input. Please enter a valid integer or 'q' to exit.")

In [ ]:
#%%memit -r 1
# If you have multiple webcams connected you need to determine which one to use
# pip install cv2_enumerate_cameras
from cv2_enumerate_cameras import enumerate_cameras
#Set search files and bounding boxes
filenameSearch=''
filenameTemplates=''

match scoreNumber:
    #Punkte
    case 1:
        filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis1DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=305
        searchBoxWidth=1990
        searchBoxHeight=2030
        ruleSet=1
    case 2:
        filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis2DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=290
        searchBoxWidth=2000
        searchBoxHeight=2030
        ruleSet=1
    # Pizzicati
    case 3:
        filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis3DPI300Gamma.jpg'
        searchBoxX=240
        searchBoxY=280
        searchBoxWidth=2000
        searchBoxHeight=2050
        ruleSet=2
    case 4:
        filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        filenameSearch='./input/LudusMusicalis4DPI300Gamma.jpg'
        searchBoxX=250
        searchBoxY=280
        searchBoxWidth=2000
        searchBoxHeight=2050
        ruleSet=2
    case 5:
        filenameSearch=webview_file_dialog()
        if filenameSearch==None:
            raise SystemExit("No File Selected")
        
        valid_rule_sets = {1, 2}
        while True:
            ruleSet_input = input("Enter the set of rules you want to adopt (1 Punkte, 2 Pizzicati) [type 'q' to quit]: ")
            if ruleSet_input.lower() in ['q', 'quit']:
                raise SystemExit("User requested exit.")
            try:
                ruleSet = int(ruleSet_input)
                if ruleSet in valid_rule_sets:
                    break
                else:
                    print("Invalid rule set. Please enter 1 for Punkte or 2 for Pizzicati.")
            except ValueError:
                print("Invalid input. Please enter a numeric value or 'q' to exit.")

        if ruleSet==2:
            filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        elif ruleSet==1:
            filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        else :
            raise SystemExit("Invalid Input")
        # ScanNewPizzicati.jpg coordinates
        searchBoxX=350
        searchBoxY=100
        searchBoxWidth=2300
        searchBoxHeight=2290

    case 6:
        print("Opening camera")

    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

if scoreNumber==6:
        
        # On Mac this package has no implementation yet...
        if platform.system() != "Darwin":
            for camera_info in enumerate_cameras(cv2.CAP_MSMF):
                print(f'{camera_info.index}: {camera_info.name}')
        else:
            print("Camera enumeration is not supported on Mac.")

        # You might want to manually set the focus of the camera, in this case you need the appropriate drivers, and making sure that multiple applications can access the camera at the same time.

        # If you pass unreasonably high values for the resolution the camera will return the highest available.
        HIGH_VALUE = 100000
        WIDTH = HIGH_VALUE
        HEIGHT = HIGH_VALUE
        # Some cameras, such as the Logitech C920, do not report their correct resolution. This particular camera is 1920x1080, but it reports 2560x1472.
        # The only solution is to manually set the correct values.
        # WIDTH = 1920
        # HEIGHT = 1080

        while True:
                cam_choice = input("Enter the webcam index you want to use [type 'q' to quit]: ")
                if cam_choice.lower() in ["q", "quit"]:
                    raise SystemExit("User requested exit.")
                try:
                    cam_choice = int(cam_choice)
                    break
                except ValueError:
                    print("Invalid input. Please enter a numeric value.")

        # Open the camera
        capture = cv2.VideoCapture(cam_choice,cv2.CAP_ANY)

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        #fourcc = cv2.VideoWriter_fourcc(*'MJPG')
        capture.set(cv2.CAP_PROP_FOURCC, fourcc)

        # Similarly to resolution a high value is replaced with the actual maximum for FPS.
        capture.set(cv2.CAP_PROP_FPS, 120)
        cfps = int(capture.get(cv2.CAP_PROP_FPS))
        print(f"Camera FPS: {cfps}")
        print(f"Camera Backend: {capture.getBackendName()}")
        print(capture.get(cv2.CAP_PROP_FOURCC))



        capture.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
        capture.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)
        width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
        print(f"Camera resolution: {width}x{height}")

        cv2.namedWindow("Acquire an image", cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty("Acquire an image", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        # On Mac this seems to be needed
        cv2.startWindowThread()

        # Get screen size
        screen_width, screen_height = get_screen_dimensions("Acquire an image")

        while True:
            ret, frame = capture.read()
            if not ret:
                print("Failed to grab frame")
                break
        
            
            # Maintain aspect ratio while fitting to screen
            aspect_ratio = width / height
            if screen_width / screen_height > aspect_ratio:
                new_height = screen_height
                new_width = int(screen_height * aspect_ratio)
            else:
                new_width = screen_width
                new_height = int(screen_width / aspect_ratio)
            
            resized_frame = cv2.resize(frame, (new_width, new_height))
            
            # Create a black canvas and center the resized frame on it
            canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
            y_offset = (screen_height - new_height) // 2
            x_offset = (screen_width - new_width) // 2
            canvas[y_offset:y_offset + new_height, x_offset:x_offset + new_width] = resized_frame
            
            # Determine the smallest dimension for squares
            min_dim = min(new_width, new_height)
            
            # Calculate larger green square properties
            large_square_size = int(0.9 * min_dim)
            large_x1 = (screen_width - large_square_size) // 2
            large_y1 = (screen_height - large_square_size) // 2
            large_x2 = large_x1 + large_square_size
            large_y2 = large_y1 + large_square_size
            
            # Calculate smaller green square properties
            small_square_size = int(0.7 * min_dim)
            small_x1 = (screen_width - small_square_size) // 2
            small_y1 = (screen_height - small_square_size) // 2
            small_x2 = small_x1 + small_square_size
            small_y2 = small_y1 + small_square_size
            
            # Draw green squares
            color = (0, 255, 0)  # Green color in BGR
            thickness = 1  # Thickness of the square
            
            # Draw the larger square
            cv2.rectangle(canvas, (large_x1, large_y1), (large_x2, large_y2), color, thickness)
            # Draw the smaller square
            cv2.rectangle(canvas, (small_x1, small_y1), (small_x2, small_y2), color, thickness)
            
            cv2.imshow("Acquire an image", canvas)
            
            k = cv2.waitKey(1)
            if k % 256 == 27:
                # ESC pressed
                print("Escape hit, closing...")
                break
            elif k % 256 == 32:
                # SPACE pressed
                print("Image acquired.")
                musicSearch = frame

        capture.release()
        # Workaround for Mac...
        cv2.waitKey(1)
        cv2.destroyAllWindows()
        # The creation of two consecutive fullscreen windows seems to be problematic on Mac. Adding a conservative delay between the two windows seems to fix the issue.
        cv2.waitKey(1000)
        #cv2.destroyWindow("Acquire an image")

        # Function to apply Gamma correction
        musicSearch = interactive_gamma_correction(musicSearch)

        musicVisualize = musicSearch.copy()

        valid_rule_sets = {1, 2}
        while True:
            ruleSet_input = input("Enter the set of rules you want to adopt (1 Punkte, 2 Pizzicati) [type 'q' to quit]: ")
            if ruleSet_input.lower() in ['q', 'quit']:
                raise SystemExit("User requested exit.")
            try:
                ruleSet = int(ruleSet_input)
                if ruleSet in valid_rule_sets:
                    break
                else:
                    print("Invalid rule set. Please enter 1 for Punkte or 2 for Pizzicati.")
            except ValueError:
                print("Invalid input. Please enter a valid integer or 'q' to exit.")

        if ruleSet==2:
            filenameTemplates = './input/LudusMusicalis3DPI300Gamma.jpg'
        elif ruleSet==1:
            filenameTemplates = './input/LudusMusicalis1DPI300Gamma.jpg'
        else :
            raise SystemExit("Invalid Input")
        # ScanNewPizzicati.jpg coordinates
        searchBoxX=350
        searchBoxY=100
        searchBoxWidth=2300
        searchBoxHeight=2290

if os.path.exists(filenameTemplates):
    musicTemplate = (ski.io.imread(filenameTemplates, as_gray=True)* 255).astype(np.uint8)
else:
    # Create an empty image with default dimensions (e.g., 2500x2500 pixels, 1 channel)
    musicTemplate = np.zeros((2500, 2500, 1), dtype=np.uint8)
musicVisualizeTemplate=musicTemplate.copy()

if scoreNumber != 6:
    if os.path.exists(filenameSearch):
        musicSearch = (ski.io.imread(filenameSearch, as_gray=True)* 255).astype(np.uint8)
        musicVisualize = musicSearch.copy()
    else:
        print(f"Error: The search file '{filenameSearch}' was not found.")
        raise SystemExit("Application stopped because the required search file is missing.")

if (scoreNumber==5 or scoreNumber==6):
    # With that we can select a Region of Interest. Press Enter or Space to confirm the selection.
    # Performance on Mac is pretty bad for drawing/interacting, but works fine after that.
    cv2.namedWindow("Select Bounding Box", cv2.WND_PROP_FULLSCREEN)
    cv2.setWindowProperty("Select Bounding Box", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    cv2.startWindowThread()

    # Call selectROI on the fullscreen window
    ROI = cv2.selectROI("Select Bounding Box", cv2.cvtColor(musicVisualize, cv2.COLOR_BGR2RGB), showCrosshair=True, printNotice=True)
    searchBoxX=ROI[0]
    searchBoxY=ROI[1]
    searchBoxWidth=ROI[2]
    searchBoxHeight=ROI[3]
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    if searchBoxHeight<100 or searchBoxWidth<100:
        raise SystemExit("Invalid Input")

colorBox = (255, 255, 0)
colorTemplate = (0, 255, 0)

We build a list of templates used for matching the various symbols.

Pizzicati contains only one symbol (a cross), but Punkte has variations in: 1 Size, 2 Empty/Full, and 3 Normal, Single Strike, Double Strike.

In [ ]:
#%%memit -r 1
valid_template_sets = {1, 2}
while True:
    template_input = input("Enter the type of templates you want to use (1 Original Files, 2 Generated templates) [type 'q' to quit]: ")
    if template_input.lower() in ['q', 'quit']:
        raise SystemExit("User requested exit.")
    try:
        templateSet = int(template_input)
        if templateSet in valid_template_sets:
            break
        else:
            print("Invalid template type. Please enter 1 for Original Files or 2 for Generated templates.")
    except ValueError:
        print("Invalid input. Please enter a valid integer or 'q' to exit.")

In [ ]:
#%%memit -r 1
# Build Templates
match ruleSet:
    #Punkte
    case 1:
        match templateSet:
            case 1:
                # Original Templates
                partDot = musicTemplate[2360:2360+82, 1214:1214+80]
                cv2.rectangle(musicVisualizeTemplate, (1214, 2360), (1214+80, 2360+82), colorTemplate, 1)
                bigWhiteDot = musicTemplate[675:675+38, 1352:1352+34]
                cv2.rectangle(musicVisualizeTemplate, (1352, 675), (1352+34, 675+38), colorTemplate, 1)
                mediumWhiteDot = musicTemplate[684:684+26, 1440:1440+24]
                cv2.rectangle(musicVisualizeTemplate, (1440, 684), (1440+24, 684+26), colorTemplate, 1)
                smallWhiteDot = musicTemplate[782:782+21, 1438:1438+16]
                cv2.rectangle(musicVisualizeTemplate, (1438, 782), (1438+16, 782+21), colorTemplate, 1)
                bigBlackDot = musicTemplate[752:752+38, 964:964+34]
                cv2.rectangle(musicVisualizeTemplate, (964, 752), (964+34,752+38), colorTemplate, 1)
                mediumBlackDot = musicTemplate[813:813+27, 960:960+26]
                cv2.rectangle(musicVisualizeTemplate, (960, 813), (960+26, 813+27), colorTemplate, 1)
                smallBlackDot = musicTemplate[824:824+22, 1008:1008+20]
                cv2.rectangle(musicVisualizeTemplate, (1008, 824), (1008+20, 824+22), colorTemplate, 1)
                mediumWhiteDoubleStrikeDot = musicTemplate[1361:1361+40, 1874:1874+43]
                cv2.rectangle(musicVisualizeTemplate, (1874, 1361), (1874+43, 1361+40), colorTemplate, 1)
                #mediumWhiteDoubleStrikeDot = music[1369:1369+26, 1884:1884+24]
                mediumWhiteSingleStrikeDot = musicTemplate[993:993+28, 348:348+34]
                cv2.rectangle(musicVisualizeTemplate, (348, 993), (348+34, 993+28), colorTemplate, 1)
                mediumBlackDoubleStrikeDot = musicTemplate[1162:1162+38, 381:381+45]
                cv2.rectangle(musicVisualizeTemplate, (381, 1162), (381+45, 1162+38), colorTemplate, 1)
                mediumBlackSingleStrikeDot = musicTemplate[1161:1161+30, 1806:1806+45]
                cv2.rectangle(musicVisualizeTemplate, (1806, 1161), (1806+45, 1161+30), colorTemplate, 1)

                plt.figure(0)
                plt.title("Part Dot")
                plt.imshow(partDot, cmap="gray")
                plt.figure(1)
                plt.title("Big White Dot")
                plt.imshow(bigWhiteDot, cmap="gray")
                plt.figure(2)
                plt.title("Medium White Dot")
                plt.imshow(mediumWhiteDot, cmap="gray")
                plt.figure(3)
                plt.title("Small White Dot")
                plt.imshow(smallWhiteDot, cmap="gray")
                plt.figure(4)
                plt.title("Big Black Dot")
                plt.imshow(bigBlackDot, cmap="gray")
                plt.figure(5)
                plt.title("Medium Black Dot")
                plt.imshow(mediumBlackDot, cmap="gray")
                plt.figure(6)
                plt.title("Small Black Dot")
                plt.imshow(smallBlackDot, cmap="gray")
                plt.figure(7)
                plt.title("Medium White Double Strike Dot")
                plt.imshow(mediumWhiteDoubleStrikeDot, cmap="gray")
                plt.figure(8)
                plt.title("Medium White Single Strike Dot")
                plt.imshow(mediumWhiteSingleStrikeDot, cmap="gray")
                plt.figure(9)
                plt.title("Medium Black Double Strike Dot")
                plt.imshow(mediumBlackDoubleStrikeDot, cmap="gray")
                plt.figure(10)
                plt.title("Medium Black Single Strike Dot")
                plt.imshow(mediumBlackSingleStrikeDot, cmap="gray")
            case 2:
                # Generated Templates
                # For some reason the grayscale conversion uses 64 bit Floats. Multi Template Matching only allows 8 and 32 bit images.
                MDTemplateGenerated = ski.io.imread('./templates/MWDTemplateGenerated.jpg', as_gray=True)
                mediumWhiteDot = (MDTemplateGenerated * 255).astype(np.uint8)
                MDTemplateGenerated = ski.io.imread('./templates/MBDTemplateGenerated.jpg', as_gray=True)
                mediumBlackDot = (MDTemplateGenerated * 255).astype(np.uint8)
                MDTemplateGenerated = ski.io.imread('./templates/WDDTemplateGenerated.jpg', as_gray=True)
                mediumWhiteDoubleStrikeDot = (MDTemplateGenerated * 255).astype(np.uint8)
                MDTemplateGenerated = ski.io.imread('./templates/WSDTemplateGenerated.jpg', as_gray=True)
                mediumWhiteSingleStrikeDot = (MDTemplateGenerated * 255).astype(np.uint8)
                MDTemplateGenerated = ski.io.imread('./templates/BDDTemplateGenerated.jpg',as_gray=True)
                mediumBlackDoubleStrikeDot = (MDTemplateGenerated * 255).astype(np.uint8)
                MDTemplateGenerated = ski.io.imread('./templates/BSDTemplateGenerated.jpg',as_gray=True)
                mediumBlackSingleStrikeDot = (MDTemplateGenerated * 255).astype(np.uint8)

                plt.figure(0)
                plt.title("Medium White Dot")
                plt.imshow(mediumWhiteDot, cmap="gray")
                plt.figure(1)
                plt.title("Medium Black Dot")
                plt.imshow(mediumBlackDot, cmap="gray")
                plt.figure(2)
                plt.title("Medium White Double Strike Dot")
                plt.imshow(mediumWhiteDoubleStrikeDot, cmap="gray")
                plt.figure(3)
                plt.title("Medium White Single Strike Dot")
                plt.imshow(mediumWhiteSingleStrikeDot, cmap="gray")
                plt.figure(4)
                plt.title("Medium Black Double Strike Dot")
                plt.imshow(mediumBlackDoubleStrikeDot, cmap="gray")
                plt.figure(5)
                plt.title("Medium Black Single Strike Dot")
                plt.imshow(mediumBlackSingleStrikeDot, cmap="gray")
            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")
            
    case 2:
        # Pizzicati
        match templateSet:
            case 1:
                # Original Templates
                partDot = musicTemplate[2348:2348+84, 1217:1217+80]
                cv2.rectangle(musicVisualizeTemplate, (1217, 2348), (1217+80, 2348+84), colorTemplate, 1)
                mediumCross = musicTemplate[1207:1207+24, 1265:1265+30]
                cv2.rectangle(musicVisualizeTemplate, (1265, 1207), (1265+30, 1207+24), colorTemplate, 1)

                plt.figure(0)
                plt.title("Part Dot")
                plt.imshow(partDot, cmap="gray")
                plt.figure(1)
                plt.title("Medium Cross")
                plt.imshow(mediumCross, cmap="gray")
            case 2:
                # Generated Templates
                # For some reason the grayscale conversion uses 64 bit Floats. Multi Template Matching only allows 8 and 32 bit images.
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated1.jpg', as_gray=True)
                mediumCross = (MCTemplateGenerated * 255).astype(np.uint8)
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated2.jpg', as_gray=True)
                mediumCross2 = (MCTemplateGenerated * 255).astype(np.uint8)
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated3.jpg', as_gray=True)
                mediumCross3 = (MCTemplateGenerated * 255).astype(np.uint8)
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated4.jpg', as_gray=True)
                mediumCross4 = (MCTemplateGenerated * 255).astype(np.uint8)
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated5.jpg', as_gray=True)
                mediumCross5 = (MCTemplateGenerated * 255).astype(np.uint8)
                MCTemplateGenerated = ski.io.imread('./templates/MCTemplateGenerated6.jpg', as_gray=True)
                mediumCross6 = (MCTemplateGenerated * 255).astype(np.uint8)
                
                plt.figure(0)
                plt.title("Medium Cross")
                plt.imshow(mediumCross, cmap="gray")
                plt.figure(1)
                plt.title("Medium Cross")
                plt.imshow(mediumCross2, cmap="gray")
                plt.figure(2)
                plt.title("Medium Cross")
                plt.imshow(mediumCross3, cmap="gray")
                plt.figure(3)
                plt.title("Medium Cross")
                plt.imshow(mediumCross4, cmap="gray")
                plt.figure(4)
                plt.title("Medium Cross")
                plt.imshow(mediumCross5, cmap="gray")
                plt.figure(5)
                plt.title("Medium Cross")
                plt.imshow(mediumCross6, cmap="gray")
            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")
        
    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

Here we display the templates, and the chosen target image with the corresponding bounding box.

In [ ]:
#%%memit -r 1
# Bounding Box
cv2.rectangle(musicVisualize, (searchBoxX, searchBoxY), (searchBoxX+searchBoxWidth, searchBoxY+searchBoxHeight), colorBox, 2)

plt.figure(11,figsize=(15, 15))
plt.title("Templates")
plt.imshow(musicVisualizeTemplate,cmap="gray")

plt.figure(12,figsize=(15, 15))
plt.title("Bounding Box")
plt.imshow(musicVisualize,cmap="gray")

In [ ]:
#%%memit -r 1
# First format the templates into a list of tuples (label, templateImage)
match ruleSet:
    case 1:
        # Punkte
        match templateSet:
            case 1:
                listTemplate = [('MWD', mediumWhiteDot), ('MBD', mediumBlackDot), ('WDD',mediumWhiteDoubleStrikeDot), ('WSD',mediumWhiteSingleStrikeDot), ('BDD',mediumBlackDoubleStrikeDot), ('BSD',mediumBlackSingleStrikeDot)]
         
                #listTemplate = [('BWD', bigWhiteDot), ('SWD', smallWhiteDot), ('BBD', bigBlackDot), ('SBD', smallBlackDot), ('MWD', mediumWhiteDot), ('MBD', mediumBlackDot),
                #               ('WDD', mediumWhiteDoubleStrikeDot), ('WSD', mediumWhiteSingleStrikeDot), ('BDD', mediumBlackDoubleStrikeDot), ('BSD', mediumBlackSingleStrikeDot)]
    
            case 2:
                listTemplate = [('MWD', mediumWhiteDot), ('MBD', mediumBlackDot), ('WDD',mediumWhiteDoubleStrikeDot), ('WSD',mediumWhiteSingleStrikeDot), ('BDD',mediumBlackDoubleStrikeDot), ('BSD',mediumBlackSingleStrikeDot)]


            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")                            
    case 2:
        # Pizzicati
        match templateSet:
            case 1:
                listTemplate = [('MC', mediumCross),]
            case 2:
                # listTemplate = [('MC1', mediumCross),('MC2', mediumCross2),('MC3', mediumCross3),('MC4', mediumCross4),('MC5', mediumCross5),('MC6', mediumCross6),]
                listTemplate = [('MC2', mediumCross2),('MC6', mediumCross6),]

            case _:
                # Anything else.
                # 
                # If `case _:` is omitted, an error will be thrown
                # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")

    case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
        raise SystemExit("Invalid Input")

We generate an augmented template list. This means that on top of the original templates we produce scaled versions between 50% and 150%, 90 degree rotations, and mirroring on X and Y axis (and both).

For every template in the original list we build 176 templates in total. This is needed as we can potentially present the score in any orientation, and when using a different image we can have a different scaling (e.g. the distance of the image from the lens).

In [ ]:
#%%memit -r 1
image_maxwh = musicSearch.shape
listTemplateSnapshot = listTemplate.copy()
listTemplate.clear()
for element in listTemplateSnapshot:
    for angles in range(0, 360, 90):
        for scales in range(50, 160, 10):
            # Scale and Rotate
            rotated = rotate(element[1], angle=angles)
            scaled, percentage = scale_image(rotated, scales, image_maxwh)
            listTemplate.append(('S '+str(angles)+' '+str(scales)+' '+element[0], scaled))
            listTemplate.append(('U '+str(angles)+' '+str(scales)+' '+element[0], np.flipud(scaled)))
            listTemplate.append(('L '+str(angles)+' '+str(scales)+' '+element[0], np.fliplr(scaled)))
            listTemplate.append(('UL '+str(angles)+' '+str(scales)+' '+element[0], np.flipud(np.fliplr(scaled))))

In [ ]:
#%%memit -r 1
# Then call the function matchTemplates setting the search box to fall inside the square

# Convert musicSearch (RGB) to grayscale (black and white) if it's not already in that format.
# If musicSearch has 3 channels then convert, otherwise assume it is already grayscale
if len(musicSearch.shape) == 3 and musicSearch.shape[2] == 3:
    musicBW = cv2.cvtColor(musicSearch, cv2.COLOR_RGB2GRAY)
else:
    musicBW = musicSearch.copy()

# Using BW Image 0.4 threshold works better, but with the original image you get too many false positives. 0.60 is more likely to work. A lower value requires longer processing time.
score_threshold = 0.40

listHits = matchTemplates(listTemplate, musicBW, score_threshold=score_threshold,method=cv2.TM_CCOEFF_NORMED, maxOverlap=(0 if ruleSet==1 else 0.1), searchBox=(searchBoxX,searchBoxY, searchBoxWidth, searchBoxHeight))

print("Found {} hits".format(len(listHits)))

In [ ]:
#%%memit -r 1
# Set the initial threshold and adjustment step
local_threshold = score_threshold
threshold_step = 0.005

# Create a fullscreen window
cv2.namedWindow("Overlay", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Overlay", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
cv2.startWindowThread()

while True:
    # Filter listHits based on its third column being >= local_threshold
    filtered_listHits = [hit for hit in listHits if hit[2] >= local_threshold]

    # Generate the overlay image with the filtered hits
    Overlay = drawBoxesOnRGB(
        musicVisualize, filtered_listHits,
        boxThickness=1, boxColor=[255, 0, 0],
        showLabel=True, labelColor=(255, 0, 0), labelScale=0.3
    )

    # Overlay the current threshold value and instructions in bright green
    instructions = f"Threshold: {local_threshold:.3f}   [Q: increase, A: decrease]   [SPACE/ENTER: confirm]"
    cv2.putText(Overlay, instructions, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    # Convert the image from RGB to BGR for proper display with OpenCV
    Overlay_BGR = cv2.cvtColor(Overlay, cv2.COLOR_RGB2BGR)

    # Get the dimensions of the fullscreen window
    screen_width, screen_height = get_screen_dimensions("Overlay")

    # Calculate the aspect ratios
    img_height, img_width = Overlay_BGR.shape[:2]
    image_aspect = img_width / img_height
    screen_aspect = screen_width / screen_height

    # Determine new dimensions to fit screen while preserving aspect ratio
    if screen_aspect > image_aspect:
        new_height = screen_height
        new_width = int(new_height * image_aspect)
    else:
        new_width = screen_width
        new_height = int(new_width / image_aspect)

    # Resize the image to these new dimensions
    preview_resized = cv2.resize(Overlay_BGR, (new_width, new_height))

    # Create a black canvas of screen dimensions and center the resized image within it
    canvas = np.zeros((screen_height, screen_width, 3), dtype=np.uint8)
    y_offset = (screen_height - new_height) // 2
    x_offset = (screen_width - new_width) // 2
    canvas[y_offset:y_offset+new_height, x_offset:x_offset+new_width] = preview_resized

    # Display the canvas in the fullscreen window
    cv2.imshow("Overlay", canvas)

    key = cv2.waitKey(0)
    if key == ord('q'):
        local_threshold = min(1.0, local_threshold + threshold_step)  # Increase threshold but cap to 1.0
    elif key == ord('a'):
        local_threshold = max(score_threshold, local_threshold - threshold_step)  # Decrease threshold but not below score_threshold
    elif key in [32, 13]:  # SPACE or ENTER confirms the selection
        break

#cv2.destroyWindow("Overlay")
cv2.waitKey(1)
cv2.destroyAllWindows()
cv2.waitKey(1)

In [ ]:
#%%memit -r 1
# Prepare the lists of symbols sorted for the 4 parts
listSymbolsLeft=sorted(filtered_listHits, key=lambda tup: (tup[1][0],tup[1][1]) )
listSymbolsRight=sorted(filtered_listHits, key=lambda tup: (tup[1][0],tup[1][1]),reverse=True )
listSymbolsTop=sorted(filtered_listHits, key=lambda tup: (tup[1][1],tup[1][0]) )
listSymbolsBottom=sorted(filtered_listHits, key=lambda tup: (tup[1][1],tup[1][0]),reverse=True )

In [ ]:
#%%memit -r 1
# Duration of the piece in seconds (1 to have it normalized between 0 and 1)
duration=1
# 1 Pixel duration in Seconds
onePixelDurLR=duration/searchBoxWidth
onePixelDurTB=duration/searchBoxHeight

# 1 Pixel in "MIDI Pitches"
pitchRange=127
onePixelPitchLR=pitchRange/searchBoxHeight
onePixelPitchTB=pitchRange/searchBoxWidth
MIDIFull=[]

In [ ]:
#%%memit -r 1
# Part 1 (Left to Right X: Time Y: Pitch)
# First we rescale both Axis
MIDILeft=[]
for x in range (len(listSymbolsLeft)):
    listCoords=list(listSymbolsLeft[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round((listSymbolsLeft[x][1][0]-searchBoxX)*onePixelDurLR,10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxHeight+searchBoxY-listSymbolsLeft[x][1][1])*onePixelPitchLR))
    MIDILeft.append((listSymbolsLeft[x][0],(listCoords),listSymbolsLeft[x][2]))
MIDIFull.append(MIDILeft)

In [ ]:
#%%memit -r 1
# Part 2 (Right to Left X: Time Y: Pitch)
# First we rescale both Axis
MIDIRight=[]
for x in range (len(listSymbolsRight)):
    listCoords=list(listSymbolsRight[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(duration-((listSymbolsRight[x][1][0]-searchBoxX)*onePixelDurLR),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxHeight+searchBoxY-listSymbolsRight[x][1][1])*onePixelPitchLR))
    MIDIRight.append((listSymbolsRight[x][0],(listCoords),listSymbolsRight[x][2]))
MIDIFull.append(MIDIRight)

In [ ]:
#%%memit -r 1
# Part 3 (Top to Bottom X: Pitch Y: Time)
# First we rescale both Axis
MIDITop=[]
for x in range (len(listSymbolsTop)):
    listCoords=list(listSymbolsTop[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(((listSymbolsTop[x][1][1]-searchBoxY)*onePixelDurTB),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxWidth+searchBoxX-listSymbolsTop[x][1][0])*onePixelPitchTB))
    MIDITop.append((listSymbolsTop[x][0],(listCoords),listSymbolsTop[x][2]))
MIDIFull.append(MIDITop)

In [ ]:
#%%memit -r 1
# Part 4 (Bottom to Top X: Pitch Y: Time)
# First we rescale both Axis
MIDIBottom=[]
for x in range (len(listSymbolsBottom)):
    listCoords=list(listSymbolsBottom[x][1]).copy()
    # 10 Decimals for time
    listCoords[0]=round(duration-((listSymbolsBottom[x][1][1]-searchBoxY)*onePixelDurTB),10)
    # Nearest Integer for MIDI Pitch
    listCoords[1]=round(((searchBoxWidth+searchBoxX-listSymbolsBottom[x][1][0])*onePixelPitchTB))
    MIDIBottom.append((listSymbolsBottom[x][0],(listCoords),listSymbolsBottom[x][2]))
MIDIFull.append(MIDIBottom)

In [ ]:
#%%memit -r 1
# Instruments and canonical MIDI range and program number
instrumentRange={
"Violin": (55,103,41, "Strings"),
"Viola": (48,91,42, "Strings"),
"Cello": (36,76,43, "Strings"),
"Double Bass": (28,67,44, "Strings"),
"Bass Guitar": (28,67,34, "Strings"),
"Acoustic Guitar": (40,88,26, "Strings"),
"Tuba": (28,58,59, "Brass"),
"Bass Trombone": (34,67,58, "Brass"),
"French Horn": (34,77,61, "Brass"),
"Trombone": (40,72,58, "Brass"),
"Trumpet": (55,82,57, "Brass"),
"Piccolo": (74,102,73, "Woodwinds"),
"Flute": (60,96,74, "Woodwinds"),
"Oboe": (58,91,69, "Woodwinds"),
"Alto Flute": (55,91,74, "Other"),
"English Horn": (52,81,70, "Other"),
"Clarinet": (50,94,72, "Woodwinds"),
"Bass Clarinet": (38,77,72, "Woodwinds"),
"Bassoon": (34,75,71, "Woodwinds"),
"Contrabassoon": (22,53,71, "Woodwinds"),
"Soprano Recorder": (72,98,75, "Other"),
"Alto Recorder": (65,91,75, "Other"),
"Tenor Recorder": (60,86,75, "Other"),
"Bass Recorder": (53,79,75, "Other"),
"Baritone Sax": (36,69,68, "Other"),
"Tenor Sax": (44,76,67, "Other"),
"Alto Sax": (49,81,66, "Other"),
"Soprano Sax": (56,88,65, "Other"),
"Glockenspiel": (79,108,10, "Pitched Percussion"),
"Xylophone": (65,108,14, "Pitched Percussion"),
"Vibraphone": (53,89,12, "Pitched Percussion"),
"Marimba": (45,96,13, "Pitched Percussion"),
"Bass Marimba": (33,81,13, "Pitched Percussion"),
"Celeste": (60,108,9, "Pitched Percussion"),
"Tubular Bells": (60,77,15, "Pitched Percussion"),
"Timpani": (40,55,48, "Pitched Percussion"),
"Harpsichord": (29,89,7, "Keyboard"),
"Harp": (24,103,47, "Other"),
"Piano": (21,108,1, "Keyboard"),
"Unpitched Percussion": (35,81,10, "Unpitched Percussion"),
}

# Dynamic and average MIDI Velocity
dynamicRange={
"ppp":(16),
"pp":(32),
"p":(48),
"mp":(64),
"mf":(80),
"f":(96),
"ff":(112),
"fff":(127),
}

We finally want to build a MIDI rendering of the score, and a "transcription" with MusicXML.

One notable situation we need to address is that when we generate a NoteOn we need to generate a corresponding NoteOff somewhere in the future.

MIDI Files encode the timing of events as a delta from the previous event.

The solution is to build a data structure that contains note off with the absolute timing. Then sort it based on the timing attribute, and finally convert to delta timing by subtracting from the timing of the current event the timing of the previous event.

In [ ]:
#%%memit -r 1
match scoreNumber:
        case 1|2:
                MIDIFileName='Punkte'+str(scoreNumber)
                noteDuration=32

        case 3|4:
                MIDIFileName='Pizzicati'+str(scoreNumber)
                noteDuration=32

        case 5|6:
                # File name
                while True:
                        user_file_name = input("Enter the name of the file to save: ")
                        if user_file_name.strip():
                                MIDIFileName = user_file_name
                                break
                        else:
                                print("Filename cannot be empty. Please enter a valid filename.")

                # Individual note duration
                while True:
                        note_duration_input = input("Enter minimum note duration (e.g 64 for 1/64th): ")
                        try:
                                noteDuration = int(note_duration_input)
                                break
                        except ValueError:
                                print("Invalid input. Please enter a valid integer for note duration.")
        case _:
        # Anything else.
        # 
        # If `case _:` is omitted, an error will be thrown
        # if `something` doesn't match any of the patterns.
                raise SystemExit("Invalid Input")  



# List to hold conversion tasks (tuples of (input, output))
conversion_tasks = []

# Tempo in BPM
performanceTempo=60
# Duration at x BPM in sec
while True:
    duration_input = input("Enter performance duration in seconds: ")
    try:
        performanceDuration = int(duration_input)
        break
    except ValueError:
        print("Invalid input. Please enter a valid integer for performance duration.")
performanceMins=performanceDuration/60
# Ticks per beat (conventionally quarter note)
ticksPerBeat=480

noteDelta=int(ticksPerBeat/(noteDuration/4))
# Compute the number of microseconds per tick (used for setting BPM) as the number of microseconds in 1 minute divided by BPM
microsecondsPerTick=int(60000000/performanceTempo)
# Conversion from 0-1 Time unit to Ticks
conversionTicks=int(ticksPerBeat*performanceTempo*performanceMins)

# List of instruments and order of appearance of the "directions"
instrumentList=['Violin','Trumpet','Flute','Piano']
trackOrder=['Left','Right','Top','Bottom']

# Create the complete MIDI File
midFull = MidiFile(ticks_per_beat=ticksPerBeat, type=1,)
for x in range(len(MIDIFull)):
        # Generate a new track
        track = MidiTrack()
        # Add it to the full file AND to a file for individual tracks
        midFull.tracks.append(track)
        midLocal=MidiFile(ticks_per_beat=ticksPerBeat, type=1,)
        midLocal.tracks.append(track)
        # Set the BPM in MIDI Track
        track.append(MetaMessage('set_tempo', tempo=microsecondsPerTick))
        # Set the instrument according to GM 1 (e.g. 46 is Pizzicato Strings)
        instrument=instrumentRange[instrumentList[x]]
        if instrument[3]=='Unpitched Percussion':
                track.append(Message('program_change', program = instrument[2], channel = 10))
        else:
                track.append(Message('program_change', program = instrument[2], channel = 0))
        pitchCorrection=(instrument[1]-instrument[0])/pitchRange
        linearizedEvents=[]
        for event in MIDIFull[x]:
                # Here we need to process individual symbols that can translate to one or more notes
                currentPitch=round((event[1][1]*pitchCorrection)+instrument[0])
                match ruleSet:
                        case 1:
                                # In Punkte symbols can generate a sequence of 1 to 3 notes for all instruments, and on top of that can generate chords for piano
                                # Punkte also alters the dynamics
                                #Parse event[0]
                                tokens=event[0].split(" ")
                                # Token[2] gives us the "size" of the symbol
                                size=int(tokens[2])
                                # Token[3] gives us the type of symbol: plain, single or double strike
                                symbol=tokens[3]

                                clusterNum=1
                                if instrument[3]=='Keyboard':
                                       # For keyboards size is mapped to note clusters
                                       clusterNum=int(((size-50.0)/100.0)*(4-1)+1)
                                       # Velocity is mapped to black/white dots
                                       match symbol:
                                                case 'MWD'|'WSD'|'WDD':
                                                        velocity=dynamicRange['pp']
                                                case 'MBD'|'BSD'|'BDD':
                                                        velocity=dynamicRange['f']
                                                case _:
                                                        # Anything else.
                                                        # 
                                                        # If `case _:` is omitted, an error will be thrown
                                                        # if `something` doesn't match any of the patterns.
                                                        raise SystemExit("Invalid Input")
                                else:
                                # For everything but keyboards size is mapped to dynamics
                                        velocity=int(((size-50.0)/100.0)*(dynamicRange['ff']-dynamicRange['pp'])+dynamicRange['pp'])

                                match symbol:
                                        case 'MWD':
                                                repetitions=1
                                                if instrument[3]=='Woodwinds':
                                                        repetitions=0
                                        case 'MBD':
                                                repetitions=1
                                                if instrument[3]=='Brass':
                                                        repetitions=0
                                        case 'WSD':
                                                repetitions=2
                                                if instrument[3]=='Woodwinds':
                                                        repetitions=0
                                        case 'BSD':
                                                repetitions=2
                                                if instrument[3]=='Brass':
                                                        repetitions=0
                                        case 'WDD':
                                                repetitions=3
                                                if instrument[3]=='Woodwinds':
                                                        repetitions=0
                                        case 'BDD':
                                                repetitions=3
                                                if instrument[3]=='Brass':
                                                        repetitions=0
                                        case _:
                                                # Anything else.
                                                # 
                                                # If `case _:` is omitted, an error will be thrown
                                                # if `something` doesn't match any of the patterns.
                                                raise SystemExit("Invalid Input")
                                for t in range(repetitions):
                                        for w in range(clusterNum):
                                                linearizedEvents.append(('note_on', currentPitch+w, velocity, int((event[1][0]*conversionTicks)+(noteDelta*(t)))))
                                                linearizedEvents.append(('note_off', currentPitch+w, 127, int(event[1][0]*conversionTicks)+(noteDelta*(t+1))))
                        case 2:
                                #Pizzicati
                                linearizedEvents.append(('note_on', currentPitch, 100, int(event[1][0]*conversionTicks)))
                                linearizedEvents.append(('note_off', currentPitch, 127, int(event[1][0]*conversionTicks)+noteDelta))

                        case _:
                                # Anything else.
                                # 
                                # If `case _:` is omitted, an error will be thrown
                                # if `something` doesn't match any of the patterns.
                                raise SystemExit("Invalid Input")  

        # Sort in ascending order of delta
        linearSortedEvents=sorted(linearizedEvents, key=lambda tup: (tup[3],tup[0]))

        for y in range(len(linearSortedEvents)):
                # Correct for Delta Time
                if (y==0):
                        track.append(Message(linearSortedEvents[y][0], note=linearSortedEvents[y][1], velocity=linearSortedEvents[y][2], time=linearSortedEvents[y][3]))
                        continue
                track.append(Message(linearSortedEvents[y][0], note=linearSortedEvents[y][1], velocity=linearSortedEvents[y][2], time=linearSortedEvents[y][3]-linearSortedEvents[y-1][3]))

        output_mid = './output/' + MIDIFileName + '_' + trackOrder[x] + '.mid'
        midLocal.save(output_mid)
        xml_file = './output/' + MIDIFileName + '_' + trackOrder[x] + '.musicxml'
        conversion_tasks.append((output_mid, xml_file))

# Save the full MIDI file and add it to conversion tasks
full_mid_file = './output/' + MIDIFileName + '_Full.mid'
midFull.save(full_mid_file)
full_xml_file = './output/' + MIDIFileName + '_Full.musicxml'
conversion_tasks.append((full_mid_file, full_xml_file))

# Convert all tasks in parallel
def convert_to_musicxml(mid_file_path, xml_file_path):
    # This function must be pickleable if we want to use ProcessPoolExecutor. Is it?
    parsed = music21.converter.parse(mid_file_path, forceSource=True, quarterLengthDivisors=[8])
    if parsed.metadata is None:
        parsed.metadata = music21.metadata.Metadata()
    parsed.metadata.title = 'Ludus Musicalis'
    parsed.metadata.composer = 'Roman Haubenstock-Ramati'
    parsed.write('musicxml', fp=xml_file_path)
    return xml_file_path

def parallel_conversion(conversion_tasks):
    # Use ThreadPoolExecutor with spawn start method
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {executor.submit(convert_to_musicxml, mid, xml): (mid, xml) 
                   for mid, xml in conversion_tasks}
        for future in concurrent.futures.as_completed(futures):
            mid_file, xml_file = futures[future]
            try:
                result = future.result()
                print(f'Conversion complete: {result}')
            except Exception as e:
                print(f'Conversion failed for {mid_file} to {xml_file}: {e}')

if __name__ == '__main__':
    # Force the spawn start method in notebooks if not already set.
    multiprocessing.set_start_method("spawn", force=True)

    parallel_conversion(conversion_tasks)